## CIA Factbook Data Analysis with SQL

The CIA factbook, also known as the [World Factbook](https://www.cia.gov/the-world-factbook/), is an annual publication of the US Central Intelligence Agency. It provides basic intelligence by summarizing information about countries and regions worldwide. The factbook contains a mix of demographic and geographic data, among many more.

<img src="./images/cover_image.png"/>

In this project, we will use SQL in Jupyter Notebook to analyze data from this from my local MySQL database. The goal is to identify the leading and lagging territories of the world based on demographic indicators like population, mortality, and fertility.

## Table of Contents
---
- [Connecting to the database](#Connecting-to-the-Database)
- [Understanding the Data](#Understanding-the-Data)
- [Exploring total population](#Exploring-Total-Population)
- [Exploring population density](#Exploring-population-density)
- [Mortality and Fertility](#Mortality-and-Fertility)
- [Countries with the fastest declining population](#Countries-with-the-fastest-declining-population)
- [Conclusion](#Conclusion)
- [Limitation](#Limitation)
- [References](#References)

## Connecting to the Database
---


















In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
%matplotlib inline

import warnings
import mysql.connector as msql
from mysql.connector import Error


conn = msql.connect(host='127.0.0.1', database= 'emmanueldb', user='root', 
                       password='Maikano456?')
with warnings.catch_warnings():
        # ignore warning for non-SQLAlchemy Connection
        # see github.com/pandas-dev/pandas/issues/45660
        warnings.simplefilter('ignore', UserWarning)
        # create pandas DataFrame from database query
def run_query(q):
    with  msql.connect(host='127.0.0.1', database= 'emmanueldb', user='root', 
                       password='Maikano456?') as conn:
        return pd.read_sql_query(q, conn)

def run_command(c):
    with msql.connect(host='127.0.0.1', database= 'emmanueldb', user='root', 
                       password='Maikano456?') as conn:
        conn.isolation_level = None
        conn.execute(c)

## Understanding the Data
---
First, let's examine the tables in our database:

In [11]:

tables= pd.read_sql_query('SHOW TABLES FROM emmanueldb', conn)
tables

C:\Users\SPECIAL AGENT\AppData\Local\Temp\ipykernel_10252\2390837674.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tables= pd.read_sql_query('SHOW TABLES FROM emmanueldb', conn)


,Tables_in_emmanueldb
0,car_details
1,cars_details_prices
2,cia_factsdata
3,female_toll
4,graduate_class
5,insurance_data
6,kpmg_customer_demography
7,kpmg_transaction
8,polling unit


#### Notes
>- There are two tables in the database, **MySQL** and **cia facts data**. 
>- The sqlite_sequence table does not contain any columns. 
>- The facts table contains information on _population, birth rate, migration rate_ and many more: this is what we need.
  
We will work with the **cia_factsdata** table henceforth.

In [3]:


#examine the first five rows in cia_facts table


cia_table ='''
SELECT *
FROM cia_factsdata
LIMIT 5
'''

run_query(cia_table)

C:\Users\SPECIAL AGENT\AppData\Local\Temp\ipykernel_12156\966990518.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(q, conn)


,id,Country,Area_total,Area_land,Area_water,Population,Population_growth,Birth_rate,Death_rate,Migration_rate
0,1,Afghanistan,652230,652230,0,38346720,0.02,35.46,12.33,-0.1
1,3,Albania,28748,27398,1350,3095344,0.00,12.69,7.31,-3.2
2,4,Algeria,2381740,2381740,0,44178884,0.01,18.52,4.32,-0.8
3,5,American Samoa,224,224,0,45443,-0.02,16.70,6.10,-29.8
4,6,Andorra,468,468,0,85560,0.00,6.88,7.92,0.0


The facts table comprises **11 columns** with intuitive names:
>- **id** - Entry row number.
>- **Country** — Name of the country.
>- **area** — The country's total area (both land and water).
>- **area_land** — The country's land area in square kilometers.
>- **area_water** — The country's water area in square kilometers.
>- **population** —  Whole number of people or inhabitants in the country.
>- **population_growth** — The country's population growth as a percentage.
>- **birth_rate** — The number of births per year per 1,000 inhabitants.
>- **death_rate** — The number of deaths per year per 1,000 inhabitants.
>- **migration_rate** — The difference between the number of persons entering and leaving the country during the year per 1,000 persons. 
  
Finally, let's check the total number of records in the facts table:

In [4]:
total_number_of_countries = '''
SELECT COUNT(*) AS num_countries,
       COUNT(DISTINCT Country) AS unique_countries
 FROM cia_factsdata
 '''

run_query(total_number_of_countries)

C:\Users\SPECIAL AGENT\AppData\Local\Temp\ipykernel_12156\966990518.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(q, conn)


,num_countries,unique_countries
0,225,225


## Exploring Total Population
---
To understand how population values are distributed in the data, we can compute the population summary statics: the minimum, mean and maximum.

In [5]:
population = '''
SELECT MIN(population) AS min_population,
       CAST(AVG(population) AS float) AS avg_population,
       MAX(population) AS max_pop
 FROM cia_factsdata
 '''
run_query(population)
         

C:\Users\SPECIAL AGENT\AppData\Local\Temp\ipykernel_12156\966990518.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(q, conn)


,min_population,avg_population,max_pop
0,5257,34955900.0,1410539758


## Lets know the countries with the least and highest population

In [8]:

least_and_highest_population = '''
                               SELECT Country, Population
                               FROM cia_factsdata
                               WHERE population = (SELECT MIN(population) FROM cia_factsdata)
                               OR population = (SELECT MAX(population) FROM cia_factsdata)
                               '''
run_query(least_and_highest_population)

C:\Users\SPECIAL AGENT\AppData\Local\Temp\ipykernel_12156\966990518.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(q, conn)


,Country,Population
0,China,1410539758
1,Saint Pierre and Miquelon,5257


The minimum population is 5257, and the maximum is over 1.4 billion—the average population centers around 35 million inhabitants. We can now identify the highest and least populated countries or entities.

### A. The top 5 countries in terms of total population
---

In [13]:

top5 = '''
       SELECT Country AS top_5_populated_countries,
       population,
       ROUND(
       CAST(population AS Float)/
       CAST((SELECT SUM(population) FROM cia_factsdata) AS Float), 3) 
       AS world_proportion
       FROM cia_factsdata
       ORDER BY 2 DESC
      LIMIT 5;
     '''
run_query(top5)

C:\Users\SPECIAL AGENT\AppData\Local\Temp\ipykernel_12156\966990518.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(q, conn)


,top_5_populated_countries,population,world_proportion
0,China,1410539758,0.179
1,India,1389637446,0.177
2,United States,337341954,0.043
3,Indonesia,277329163,0.035
4,Pakistan,242923845,0.031


>- China is the world's most populated country, with about 1.37 billion people. India follows behind with about 1.26 billion people. **36%** of the world's total population comes from China (17.9%) and India (17.7%) alone.
>- The United States (4.3% of the world total), Indonesia (3.5%), and Pakistan (3.1%) follow far behind and constitute a combined **10%** of the world's population. 
>- Altogether the top 5 countries contribute **46%** to the global population.

### B. Countries with the smallest total population
---

In [20]:
least5 = '''
       SELECT Country AS least_5_populated_countries,
       population
       FROM cia_factsdata
       ORDER BY 2
      LIMIT 5;
     '''
run_query(least5)

C:\Users\SPECIAL AGENT\AppData\Local\Temp\ipykernel_12156\966990518.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(q, conn)


,least_5_populated_countries,population
0,Saint Pierre and Miquelon,5257
1,Montserrat,5414
2,Saint Barthelemy,7103
3,"Saint Helena, Ascension, and Tristan da Cunha",7925
4,Cook Islands,8128


>- The least populated territories are the Saint Pierreand Miquelon(5257 inhabitants), Montserrat (5414), Saint Barthelemy(7103), Saint Helena, Ascension, and Tristan da Cunha	(7925), and Cook Islands (8128).


## Exploring population density
---

<img src='./images/population_density.png'/>

[National Geographic](https://education.nationalgeographic.org/resource/population-density) puts this in simple terms. Population density is the concentration of individuals within a species in a specific geographic locale. Population density data can be used to quantify demographic information and to assess relationships with ecosystems, human health, and infrastructure.

Our data provides variables that we can use to estimate population density as follows:
$$ 
Population\ density = \frac{Total\ population}{Land\ area\ (km^2)}
$$

### A. Countries with the highest population density


In [25]:
high_population_density = '''
SELECT country, area_land, population,
       CAST(
            CAST(population AS Float) / 
              CAST(area_land AS Float) 
           AS Float) AS population_density
FROM cia_factsdata
WHERE area_land > 50000
ORDER BY 4 DESc
LIMIT 10
'''
run_query(high_population_density)

C:\Users\SPECIAL AGENT\AppData\Local\Temp\ipykernel_12156\966990518.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(q, conn)


,country,area_land,population,population_density
0,Bangladesh,130170,165650475,1272.570
1,"Korea, South",96920,51844834,534.924
2,India,2973193,1389637446,467.389
3,Philippines,298170,114597229,384.335
4,Sri Lanka,64630,23187516,358.773
5,Japan,364485,124214766,340.795
6,Vietnam,310070,103808319,334.790
7,Pakistan,770875,242923845,315.127
8,United Kingdom,241930,67791400,280.211
9,Nigeria,910768,225082083,247.134


>- Bangladesh leads in terms of high population and low land area with a population density of 1,272 people per square km. Bangladesh's population density is over two times the values recorded for 2nd-placed South Korea. India, the Philippines, and Sri Lanka follow behind.

### B. Countries with the lowest population density
---

In [26]:
least_population_density= 
'''
SELECT country, area_land, population,
       CAST(
            CAST(population AS Float) / 
              CAST(area_land AS Float) 
           AS Float) AS population_density
FROM cia_factsdata
WHERE area_land > 50000
ORDER BY 4 
LIMIT 10
'''
run_query(least_population_density)

C:\Users\SPECIAL AGENT\AppData\Local\Temp\ipykernel_12156\966990518.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(q, conn)


,country,area_land,population,population_density
0,Greenland,2166086,57792,0.02668
1,Mongolia,1553556,3227863,2.07773
2,Namibia,823290,2727409,3.31282
3,Australia,7682300,26141369,3.40281
4,Iceland,100250,357603,3.56711
5,Guyana,196849,789683,4.01162
6,Mauritania,1030700,4161925,4.03796
7,Suriname,156000,632638,4.05537
8,Libya,1759540,7137931,4.05670
9,Canada,9093507,38232593,4.20438


> <img src='./images/greenland_map.png' style='float: left;'/>
<div style='clear:both;'></div>

>- **Greenland** remains the world's least-densely populated territory. With a land area of over 2 million square kilometers and only about 58,000 inhabitants, Greenland looks like the number-one recommended spot for those who want to feel like they've left all of humanity behind. 
>- Based on the new criteria, Mongolia, Namibia, Australia, and Mauritania occupy the 2nd to 5th places, respectively.
  
## Mortality and Fertility
---
 
Two metrics determine the change in the world population: the number of babies born and people dying. [In 2021](https://statisticstimes.com/demographics/world-death-and-birth-rate.php), the crude death rate for the world was 7.64 deaths per thousand population, and the crude birth rate for the world is 17.76 births per thousand. These figures indicate a ratio of about 2.3 births for every death.

Let's query our data for the same information:

In [41]:
world_mortality_and_fertilty_rate = 
'''
SELECT ROUND(AVG(birth_rate),2) AS Average_Birth_Rate, 
ROUND(AVG(death_rate),2) AS Average_Death_Rate,
ROUND(AVG (birth_rate/death_rate),2) as birth_to_death_ratio
FROM cia_factsdata
'''
run_query(world_mortality_and_fertilty_rate)


C:\Users\SPECIAL AGENT\AppData\Local\Temp\ipykernel_12156\966990518.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(q, conn)


,Average_Birth_Rate,Average_Death_Rate,birth_to_death_ratio
0,17.63,7.5,2.78


Our data reveals a similar pattern, with about **2.78** births for every death.

### A. Countries with the highest birth rates
---

In [42]:
#countries with highest birth rates
highest_birth_rate = 
'''
SELECT country, birth_rate
FROM cia_factsdata
 ORDER BY 2 DESC
 LIMIT 10;
 '''
run_query(highest_birth_rate)

C:\Users\SPECIAL AGENT\AppData\Local\Temp\ipykernel_12156\966990518.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(q, conn)


,country,birth_rate
0,Niger,47.08
1,Angola,41.80
2,Benin,41.15
3,Mali,41.07
4,Uganda,40.94
5,Chad,40.45
6,"Congo, Democratic Republic of the",40.08
7,Somalia,37.98
8,Mozambique,37.47
9,Liberia,36.64


>- Niger has the highest birth rate in the world at 47.08 average annual births per 1,000 people per year. Angola (41.80), Benin (41.15), Mali (41.07), and Uganda (40.94) follow closely behind. All countries in the top-10 list belong to Africa. Africa's dominant birth rate is further emphasized in the figure below:<br><br>
> <img src='https://www.geocurrents.info/wp-content/uploads/2016/02/TFR-2010-2015-World-Map.png'
     width="600" height="400" style='float: left;'/>
<div style='clear:both;'></div><br>

Nowadays, large population growth in a country is either associated with a poor economy, lack of awareness and preventive measures, or poor living conditions. Lack of access to birth control and contraception in third-world countries also contributes to higher fertility rates. [Statistics](https://www.statista.com/statistics/280190/fertility-rate-per-woman-worldwide-by-income-group/) have shown that the fertility rate per woman in low-income countries is twice as high as the global average. As a result, 10 out of the top-10 countries with the highest birth rates are in Africa, as seen above.

### B. Countries with the lowest birth rates
---

In [44]:
#lowest birth rate
lowest_birth_rate ='''
SELECT country, birth_rate
FROM cia_factsdata
 ORDER BY 2
 LIMIT 10;
 '''
run_query(lowest_birth_rate)

C:\Users\SPECIAL AGENT\AppData\Local\Temp\ipykernel_12156\966990518.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(q, conn)


,country,birth_rate
0,Saint Pierre and Miquelon,6.47
1,Monaco,6.66
2,Andorra,6.88
3,"Korea, South",6.92
4,Italy,6.95
5,Japan,6.95
6,Spain,7.13
7,Taiwan,7.39
8,Greece,7.61
9,Puerto Rico,7.87


>- On the other side of the scale, the CIA estimates Saint pierre and Miquelon has the lowest birth rate in the world at 6.47 average annual births per 1,000 people per year. The small population size of [Saint Pierre](https://en.wikipedia.org/wiki/Saint-Pierre,_Saint_Pierre_and_Miquelon) makes it easy to understand its presence on the list. With close margins,  Monaco (6.66), Andorra (6.88), and South Korea (6.92).
This [Wikipedia article](https://en.wikipedia.org/wiki/Total_fertility_rate) provides some insights into the low birth rates recorded in Asian regions (Japan, South Korea, Taiwan, and Singapore). Aging and socio-economic challenges are described as the major reasons for these results.

### C. Countries with the highest death rates
---

In [47]:
#countries with highest death rate

high_death_rates = '''
SELECT country, death_rate
  FROM cia_factsdata
 ORDER BY 2 DESC
 LIMIT 10
'''

run_query(high_death_rates)

C:\Users\SPECIAL AGENT\AppData\Local\Temp\ipykernel_12156\966990518.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(q, conn)


,country,death_rate
0,Serbia,16.39
1,Romania,15.26
2,Lithuania,15.12
3,Latvia,14.65
4,Bulgaria,14.41
5,Ukraine,13.77
6,Russia,13.36
7,Estonia,13.10
8,Croatia,12.88
9,Hungary,12.88


### D. Countries with the lowest death rates
---

In [5]:
low_death_rate = '''
SELECT Country, death_rate
  FROM cia_factsdata
 ORDER BY 2
 LIMIT 10;
 '''
run_query(low_death_rate)

C:\Users\SPECIAL AGENT\AppData\Local\Temp\ipykernel_10252\966990518.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(q, conn)


,Country,death_rate
0,Qatar,1.42
1,United Arab Emirates,1.56
2,Kuwait,2.25
3,Bahrain,2.82
4,Gaza Strip,2.91
5,Oman,3.23
6,West Bank,3.40
7,Saudi Arabia,3.42
8,Libya,3.45
9,Jordan,3.45


- Qatar has the lowest mortality rate in the world at 1.42 deaths per 1,000 people. The UAE (1.56), Kuwait (2.25), and Bahrain (2.28) follow behind. These countries are renowned for their improved health care system, technologically advanced facilities, and ability to deliver some of the world's best patient care.

Here is another interesting pattern: **Six** of the ten countries on this list (Saudi Arabia, Kuwait, the United Arab Emirates, Qatar, Bahrain, and Oman) belong to the [Gulf Cooperation Council (GCC)](https://en.wikipedia.org/wiki/Gulf_Cooperation_Council). In fact, **all members** of the council are present on the list. The GCC is a political and economic alliance of the six aforementioned Middle Eastern countries.

<img src='./images/gcc.webp'
     width="500" height="400" style='float: left;'/>
<div style='clear:both;'></div><br>
     
Arguably, the [most important](https://www.britannica.com/topic/Gulf-Cooperation-Council) goal of the GCC lies in Article 4 of its charter, which states that the alliance was formed to strengthen relations among its member countries and to promote cooperation among the countries' citizens. This alliance may hint at the positive effect of the countries on one another.

### E. Countries with the highest birth-to-death ratio
---
 
Birth to death ratio can be calculated with the expression:

$$ 
Birth\ to\ death\ ratio = \frac{birth\ rate}{death\ rate}
$$


In [6]:
#Countries with highest birth_to_death ratio
birthdeath_ratio ='''
SELECT Country, birth_rate, death_rate,
       ROUND((birth_rate / death_rate), 2) AS birth_to_death_ratio
  FROM cia_factsdata
 ORDER BY 4 DESC
 LIMIT 10;
 '''
run_query(birthdeath_ratio)

C:\Users\SPECIAL AGENT\AppData\Local\Temp\ipykernel_10252\966990518.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(q, conn)


,Country,birth_rate,death_rate,birth_to_death_ratio
0,Gaza Strip,27.67,2.91,9.51
1,Uganda,40.94,5.02,8.16
2,Kuwait,17.78,2.25,7.90
3,West Bank,24.42,3.40,7.18
4,United Arab Emirates,10.81,1.56,6.93
5,Oman,22.11,3.23,6.85
6,Qatar,9.33,1.42,6.57
7,Tanzania,33.30,5.09,6.54
8,Jordan,22.58,3.45,6.54
9,Iraq,24.70,3.90,6.33


>- The presence of Gulf Cooperation territories like Kuwait, the UAE, Oman, and Qatar is expected, considering the low death rate in these regions.
  
### F. Countries with the lowest birth-to-death ratio
---

In [7]:
#countries with low birth_to_death ratio
low_birthdeath_ratio= '''
SELECT Country, birth_rate, death_rate,
       ROUND((birth_rate / death_rate), 2) AS birth_to_death_ratio
  FROM cia_factsdata
 ORDER BY 4
 LIMIT 10;
 '''

run_query(low_birthdeath_ratio)

C:\Users\SPECIAL AGENT\AppData\Local\Temp\ipykernel_10252\966990518.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(q, conn)


,Country,birth_rate,death_rate,birth_to_death_ratio
0,Serbia,8.92,16.39,0.54
1,Bulgaria,8.05,14.41,0.56
2,Romania,8.76,15.26,0.57
3,Saint Pierre and Miquelon,6.47,11.22,0.58
4,Latvia,8.73,14.65,0.60
5,Japan,6.95,11.59,0.60
6,Lithuania,9.26,15.12,0.61
7,Italy,6.95,11.31,0.61
8,Monaco,6.66,10.70,0.62
9,Greece,7.61,12.04,0.63


>- Most countries and territories on the list are located in Europe. The compounded effect of decreasing birth rates and increasing deaths may pose a potential risk. The ratio indicates a struggle in Europe to achieve a single birth for every ten deaths. An article emphasizing a similar concern can be found [here](https://www.theguardian.com/world/2015/aug/23/baby-crisis-europe-brink-depopulation-disaster).

The struggle to maintain corresponding birth rates for every death in Europe might be made clearer when compared to other continents. [Wikimedia commons](https://commons.wikimedia.org/wiki/File:Birth_rate_vs_Death_rate,_OWID.svg) conducted a similar analysis of birth and death rates across the world's countries in 2016; the results are enlightening.

<img src='./images/birth_vs_death.svg'
     width="700" height="500" style='float: left;'/>
<div style='clear:both;'></div><br>

We can tell from the chart above; when compared to other continents, Europe recorded higher than average death rates and lower than average birth rates. An inherent message here is that several countries appear to be **shrinking** from a population perspective. We can identify countries with the fastest declining populations to shed some light on this alarming discovery.

## Countries with the fastest declining population
---
  
To identify the fastest declining populations, we select countries that meet the following criteria:
- A birth-to-death ratio of less than one (indicates the absence of natural increase).
- Little to no population growth (population growth of less than 1%).
- Very low immigration levels (migration rate of less than one).

In [9]:
#Countries with the fastest declining population

declining_population = '''
SELECT Country,
       ROUND((birth_rate / death_rate), 2) AS birth_death_ratio, 
       population_growth,
       migration_rate
  FROM cia_factsdata
  WHERE (birth_rate/Death_rate) < 1
     AND population_growth < 1
     AND migration_rate < 1
 ORDER BY 4, 2, 3;
 '''

run_query(declining_population)


C:\Users\SPECIAL AGENT\AppData\Local\Temp\ipykernel_10252\966990518.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(q, conn)


,Country,birth_death_ratio,population_growth,migration_rate
0,Puerto Rico,0.81,-0.01,-11.9
1,Moldova,0.82,-0.01,-8.9
2,Saint Pierre and Miquelon,0.58,-0.01,-7.2
3,Latvia,0.60,-0.01,-5.2
4,Lithuania,0.61,-0.01,-4.5
5,Romania,0.57,-0.01,-4.4
6,Estonia,0.67,-0.01,-2.8
7,Croatia,0.67,-0.01,-0.5
8,Bosnia and Herzegovina,0.82,0.00,-0.4
9,Bulgaria,0.56,-0.01,-0.3


## Conclusion
---

Throughout this project, we used the CIA World factbook data to identify the leading and lagging territories of the world. Our analysis is based on demographic indicators like population, mortality, and fertility.

>- Accounting for 46% of the world's total population, China, India, the United States, Indonesia, and Pakistan are the most populated countries. The least populated world locations are majorly Islands and dependencies.<br>
>- Due to their small land area, Macau, Monaco, Singapore, and Hong Kong are more densely populated than the rest of the world. However, when considering larger land areas, Bangladesh, South Korea, and India are the most densely populated countries.<br>
>- Fertility is highest in Africa and lowest in Europe and Asia. Some African countries (Lesotho, Guinea-Bissau) and Eastern European countries (Bulgaria, Lithuania, Serbia) record the highest death rates. The Gulf cooperation countries of the Middle-East record the lowest death rates.<br>
>- Population is rapidly declining in East and South-East Europe. This occurrence is due to reducing birth rates, increasing deaths, and emigration to other countries. Japan is also experiencing a population decline despite the positive forecasts for the country.
  
## Limitation
---
  
- It appears that the CIA collated this data between 2016 and 2020. As a result, our findings are limited to the period the data was published. They may not reflect the current state of the world's countries.

## References
---
  
- Britanica: [The Gulf Cooperation Council](https://www.britannica.com/topic/Gulf-Cooperation-Council)
- Flowing Data: [Japan fertility rate forecasts versus reality](https://flowingdata.com/2015/01/08/japan-fertility-rate-forecasts-versus-reality/).
- National Geographic: [Population density](https://education.nationalgeographic.org/resource/population-density).
- Statista: [Fertility rate per woman worldwide](https://www.statista.com/statistics/280190/fertility-rate-per-woman-worldwide-by-income-group/).
- Statistics Times: [World death and birth rate](https://statisticstimes.com/demographics/world-death-and-birth-rate.php).
- The Atlantic: [The Mystery of Why Japanese People Are Having So Few Babies](https://www.theatlantic.com/business/archive/2017/07/japan-mystery-low-birth-rate/534291/).
- The Boorgen Project: [Partnerships needed to address top diseases in Guinea Bissau](https://borgenproject.org/top-diseases-in-guinea-bissau/#:~:text=The%20leading%20cause%20of%20death,significantly%20reduce%20their%20negative%20effects).
- The Global Economy: [Country information index](https://www.theglobaleconomy.com/economies/).
- The Guardian: [Europe needs many more babies to avert a population disaster](https://www.theguardian.com/world/2015/aug/23/baby-crisis-europe-brink-depopulation-disaster).
- The Visual Capitalist: [4 Historical Maps that Explain the USSR](https://www.visualcapitalist.com/4-historical-maps-that-explain-the-ussr/)
- The Visual Capitalist: [Ranked: The 20 Countries With the Fastest Declining Populations](https://www.visualcapitalist.com/ranked-the-20-countries-with-the-fastest-declining-populations/).
- Wikimedia Commons: [Birth rate vs Death rate 2016](https://commons.wikimedia.org/wiki/File:Birth_rate_vs_Death_rate,_OWID.svg).
- Wikipedia: [Saint Pierre and Miquelon](https://en.wikipedia.org/wiki/Saint-Pierre,_Saint_Pierre_and_Miquelon).
- Wikipedia: [The World Factbook](https://en.wikipedia.org/wiki/The_World_Factbook).
- World Atlas: [Countries and dependencies of the world](https://www.worldatlas.com/geography/dependencies-and-territories-of-the-world.html).
- World Atlas: [Least densely populated places](https://education.nationalgeographic.org/resource/population-density).
- Worldometer: [World population by year](https://www.worldometers.info/world-population/world-population-by-year/).
- World Population Review: [Antartica's population](https://worldpopulationreview.com/continents/antarctica-population).